In [13]:
%load_ext autoreload
%autoreload 2

from loader import Data, load_data_from_json
import numpy as np

import pandas as pd
from model import Model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
data: Data = load_data_from_json("../data/test.json")

In [15]:
print(data)

Data(N=5, E=1, Q=1, P=1, _qualifications=[[0]], _conges=[[]], _duree=[[7]], _gain=[20], _due_date=[2], _penalty=[3], _employees_names=['Olivia'], _qualifications_names=['A'], _projects_names=['Job1'], M=10000000.0)


In [16]:
m = Model(data)

In [17]:
m.constraint_cant_work_more_than_needed()
m.constraint_cant_work_on_days_off()
m.constraint_no_qual_work()
m.constraint_one_project_per_day()

In [18]:
m.determine_affecte()
m.determine_debute()
m.determine_realise()

In [19]:
m.objective_max_gain()

In [20]:
m.update()

In [21]:
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 7 4700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 18 rows, 17 columns and 57 nonzeros
Model fingerprint: 0x6039af3a
Variable types: 0 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [3e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Found heuristic solution: objective -6.0000000
Presolve removed 18 rows and 17 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: -6 
No other solutions better than -6

Optimal solution found (tolerance 1.00e-04)
Best objective -6.000000000000e+00, best bound -6.000000000000e+00, gap 0.0000%


In [22]:
def get_completion_day(p):
    for j in range(m.data.N):
        if m.realise(p, j).X == 1:
            return j - 1
    return None


def get_work(p):
    df = pd.DataFrame()
    for e in range(m.data.E):
        work_done = []
        for j in range(m.data.N):
            work = []
            for q in range(m.data.Q):
                work.append(m.projet(e, j, p, q).X)
            has_worked = 1 in work
            if not has_worked:
                if j in m.data.conges(e):
                    work_done.append("X")
                else:
                    work_done.append(" ")
            else:
                work = np.argmax(work)
                work = m.data.qualification_name(work)
                work_done.append(work)
        df[m.data.employee_name(e)] = work_done

    due = []
    for j in range(m.data.N):
        if j == m.data.due_date(p):
            due.append("X")
        else:
            due.append("")

    completed = []
    completion_day = get_completion_day(p)
    for j in range(m.data.N):
        if j == completion_day:
            completed.append("X")
        else:
            completed.append("")

    df["Due_Date"] = due
    df["Completed"] = completed

    return df

In [23]:
for p in range(m.data.P):
    qualis = []
    for q in range(m.data.Q):
        qualis.append((m.data.qualification_name(q), m.data.duree(p, q)))
    print("Work on", m.data.projet_name(p), qualis)
    print(get_work(p))
    completion_day = get_completion_day(p)
    if completion_day != None:
        print("Project completed, gained", m.data.gain(p))
        if m.data.due_date(p) < completion_day:
            print("Project late", completion_day - m.data.due_date(p), "day, lost", m.data.penalite(p, completion_day))
        print("Total gain", m.data.gain(p) - m.data.penalite(p, completion_day))
    else:
        print("Project not completed")
        print("Project late", m.data.N - 1 - m.data.due_date(p), "day, lost", m.data.penalite(p, m.data.N - 1))
        print("Total gain", -m.data.penalite(p, m.data.N - 1))

    print()

print(m.model.ObjVal)

Work on Job1 [('A', 7)]
  Olivia Due_Date Completed
0      A                   
1      A                   
2      A        X          
3      A                   
4      A                   
Project not completed
Project late 2 day, lost 3
Total gain -3

-6.0


In [24]:
print(m.model.getVars())

[<gurobi.Var [PROJET]employe:Olivia_projet:Job1_qual:A_jour:0 (value 1.0)>, <gurobi.Var [PROJET]employe:Olivia_projet:Job1_qual:A_jour:1 (value 1.0)>, <gurobi.Var [PROJET]employe:Olivia_projet:Job1_qual:A_jour:2 (value 1.0)>, <gurobi.Var [PROJET]employe:Olivia_projet:Job1_qual:A_jour:3 (value 1.0)>, <gurobi.Var [PROJET]employe:Olivia_projet:Job1_qual:A_jour:4 (value 1.0)>, <gurobi.Var [Realise]projet:Job1_jour:0 (value 0.0)>, <gurobi.Var [Realise]projet:Job1_jour:1 (value 0.0)>, <gurobi.Var [Realise]projet:Job1_jour:2 (value 0.0)>, <gurobi.Var [Realise]projet:Job1_jour:3 (value 0.0)>, <gurobi.Var [Realise]projet:Job1_jour:4 (value 0.0)>, <gurobi.Var [Realise]projet:Job1_jour:5 (value 0.0)>, <gurobi.Var [Affecte]employe:Olivia_projet:Job1 (value 1.0)>, <gurobi.Var [Debute]projet:Job1_jour:0 (value 1.0)>, <gurobi.Var [Debute]projet:Job1_jour:1 (value 1.0)>, <gurobi.Var [Debute]projet:Job1_jour:2 (value 1.0)>, <gurobi.Var [Debute]projet:Job1_jour:3 (value 1.0)>, <gurobi.Var [Debute]projet